In [28]:
import sys, os, math
import numpy as np
sys.path.insert(0,os.path.expanduser('~/git/pyzx')) # git version
sys.path.insert(0,'/workspaces/pyzx')
import pyzx as zx
from pyzx import print_matrix
from pyzx.basicrules import *
import time
from fractions import Fraction
import random
from pyzx.gflow import gflow
from pyzx.utils import vertex_is_zx
# zx.settings.drawing_backend = 'matplotlib'

Z = zx.VertexType.Z
X = zx.VertexType.X
B = zx.VertexType.BOUNDARY
SE = zx.EdgeType.SIMPLE
HE = zx.EdgeType.HADAMARD

In [ ]:
def transpose(p):
    pt = dict()
    for k,s in p.items():
        for v in s:
            if v not in pt: pt[v] = set()
            pt[v].add(k)
    return pt
def extend(g, p):
    pe = dict()
    for k,s in p.items():
        pe[k] = s.copy()
        for v in s:
            pe[k] |= {v for v in g.neighbors(v) if vertex_is_zx(g.type(v))}
    return pe
def is_dag(p):
    pt = transpose(p)
    for k,s in p.items():
        for v in s:
            if k != v and k in pt and v in pt[k]:
                return False
    return True


In [42]:
random.seed(1337)
g = zx.generate.cliffordT(5, 400, p_t=0.2)
zx.draw(g)
g

Graph(541 vertices, 657 edges)

In [43]:
g1 = g.copy()
zx.full_reduce(g1)
g1.normalize()
g1 = g1.copy()
zx.to_rg(g1)
# g2 = g1.copy()
# for e in list(g2.edges()): zx.hrules.had_edge_to_hbox(g2, e)
zx.draw(g1, labels=True)

In [50]:
o, p = gflow(g1, focus=True, reverse=True, pauli=True)
print(o)
print(p)
ep = extend(g1, p)
print(ep)
is_dag(ep)

{5: 14, 6: 14, 7: 14, 8: 14, 13: 14, 9: 13, 10: 13, 11: 13, 58: 13, 12: 12, 15: 12, 17: 12, 18: 12, 57: 12, 59: 12, 14: 11, 16: 11, 19: 11, 20: 10, 21: 10, 22: 10, 23: 9, 24: 9, 25: 9, 27: 9, 26: 8, 28: 8, 29: 8, 30: 8, 31: 7, 40: 7, 62: 7, 32: 6, 33: 6, 34: 5, 35: 5, 36: 5, 37: 4, 38: 3, 39: 2, 41: 2, 42: 2, 44: 2, 60: 2, 61: 2, 43: 1, 45: 1, 46: 1, 47: 1, 48: 1, 51: 1, 63: 1, 49: 0, 50: 0}
{9: {8, 6}, 10: {8, 5, 6}, 11: {6}, 58: {8, 12, 5, 6}, 12: {58}, 15: {8, 10, 58}, 17: {5, 6, 8, 13, 58}, 18: {10, 58, 6, 7}, 57: {5, 6, 7, 10, 13, 14}, 59: {6, 8, 10, 16, 58}, 14: {57}, 16: {59}, 19: {10, 11, 58, 15}, 20: {19, 7}, 21: {6, 7, 9, 11, 13, 15, 17, 19, 58}, 22: {7, 9, 11, 17, 18, 19, 57, 59}, 23: {7, 11, 13, 17, 18, 20, 58, 59}, 24: {5, 15, 17, 19, 20, 21, 57, 59}, 25: {7, 8, 9, 15, 17, 18, 20, 22, 57, 59}, 27: {5, 9, 10, 17, 18, 19, 21, 22, 57, 58}, 26: {5, 6, 7, 13, 17, 18, 20, 21, 23, 58}, 28: {5, 9, 10, 11, 15, 19, 21, 22, 23, 25, 58, 57}, 29: {5, 6, 7, 8, 9, 18, 23}, 30: {6, 9, 10,

False

In [ ]:
o,p = gflow(g1, focus=True, reverse=False, pauli=True)
pt = transpose(p)
print(o)
print(pt)

{48: 14, 49: 14, 51: 14, 44: 14, 45: 14, 41: 13, 46: 13, 60: 13, 63: 13, 39: 12, 40: 12, 42: 12, 47: 12, 50: 12, 61: 12, 35: 11, 38: 11, 43: 11, 33: 10, 36: 10, 37: 10, 27: 9, 34: 9, 26: 8, 30: 8, 31: 8, 32: 8, 24: 7, 29: 7, 62: 7, 28: 6, 21: 5, 23: 5, 25: 5, 20: 4, 22: 4, 9: 3, 17: 3, 18: 3, 19: 3, 57: 3, 59: 3, 7: 2, 11: 2, 13: 2, 14: 2, 15: 2, 16: 2, 10: 1, 58: 1, 5: 0, 6: 0, 8: 0, 12: 0}
{49: {6, 7, 8, 9, 15, 17, 18, 19, 20, 21, 23, 30, 32, 35, 36, 37, 38, 39, 40, 41, 46, 57, 59}, 45: {32, 33, 5, 6, 7, 40, 41, 9, 19, 20, 21, 22, 23, 57, 26, 27, 61, 31}, 47: {60}, 50: {63}, 48: {6, 7, 8, 17, 19, 22, 24, 27, 28, 29, 30, 33, 37, 38, 39, 40, 57, 61, 62}, 51: {6, 7, 8, 9, 10, 11, 19, 20, 23, 24, 25, 27, 28, 29, 32, 33, 35, 36, 37, 38, 39, 40, 57, 58, 59, 61}, 63: {6, 7, 8, 9, 11, 13, 15, 17, 23, 24, 25, 26, 28, 29, 30, 32, 36, 37, 39, 50, 58, 61}, 41: {6, 7, 9, 11, 13, 15, 17, 19, 24, 26, 27, 28, 30, 31, 32, 33, 34, 38, 40, 58}, 44: {33, 35, 38, 40, 59, 10, 13, 15, 17, 19, 20, 57, 26, 2